In [2]:
# Change directory to cadmesh repo until conda/pip deployment is ready
import os
os.chdir("/home/sebastian/Projects/ABC/cadmesh/")

In [3]:
from cadmesh import process_step_folder
results = process_step_folder(data_path="data/abc_0000", output_path="data/results/abc_0000", log_path="data/log_abc", file_pattern="*/*.step", file_range=[800, -1])

In [ ]:
from cadmesh import analyse_log_files
analyse_log_files("data/log_abc/*.log")

In [19]:
from yaml import CLoader
import yaml

def parse_geometry_files(path):
    files = glob.glob(path)
    print(len(files))
    curve_types = []
    surf_types = []
    for file in tqdm(files):
        try:
            with open(file, "r") as fi:
                parts = yaml.load(fi, Loader=CLoader)

            if type(parts) == dict:
                for p in parts["parts"]:
                    for c3d in p["3dcurves"]:
                        curve_types.append(c3d["type"])
                    for s3d in p["surfaces"]:
                        surf_types.append(s3d["type"])
        except:
            continue
    
    return curve_types, surf_types
                
ct, st = parse_geometry_files("data/results_conv/*_geo.yaml")

992


In [32]:
counts_c = {"Line": 0, "BSpline": 0, "Circle": 0, "Ellipse": 0, "Other": 0}
for c in ct:
    counts_c[c] += 1
    
counts_s = {"Plane": 0, "Cylinder": 0, "Cone": 0, "Torus": 0, "Other": 0, "Revolution": 0, "BSpline": 0, "Sphere": 0, "Extrusion": 0, "Offset": 0}
for c in st:
    counts_s[c] += 1
print(counts_c, counts_s)

{'Line': 291341, 'BSpline': 188704, 'Circle': 189288, 'Ellipse': 10865, 'Other': 3711} {'Plane': 113428, 'Cylinder': 75372, 'Cone': 17282, 'Torus': 11986, 'Other': 0, 'Revolution': 3671, 'BSpline': 30706, 'Sphere': 3491, 'Extrusion': 2605, 'Offset': 136}
